In [1]:
import pandas as pd
import os

DATAPATH = "../data"

In [3]:
mollib_25 = pd.read_csv(os.path.join(DATAPATH, "zairachem_outputs", "output_mollib_40e_2.5.csv"))
mollib_1 = pd.read_csv(os.path.join(DATAPATH, "zairachem_outputs", "output_mollib_40e_1.csv"))

data6_25 = pd.read_csv(os.path.join(DATAPATH, "zairachem_outputs", "output_data6_2.5.csv"))
data6_1 = pd.read_csv(os.path.join(DATAPATH, "zairachem_outputs", "output_data6_1.csv"))

eosi90_25 = pd.read_csv(os.path.join(DATAPATH, "zairachem_outputs", "output_eosi_s4_candidates_90_2.5.csv"))
#eosi90_1 = pd.read_csv(os.path.join(DATAPATH, "zairachem_outputs", "output_eosi_s4_candidates_90_1.csv"))


In [13]:
#check different thresholds
print(len(data6_25[data6_25["clf_ex1_bin"]==1]))
print(len(data6_1[data6_1["clf_ex1_bin"]==1]))
print(len(mollib_25[data6_25["clf_ex1_bin"]==1]))
print(len(mollib_1[data6_1["clf_ex1_bin"]==1]))

8952
1920
388
90


C:\Users\Gemma\Anaconda3\envs\malariaOSM\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.
C:\Users\Gemma\Anaconda3\envs\malariaOSM\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """


In [17]:
#check both clf and regression coincide
print(len(data6_1[(data6_1["clf_ex1_bin"]==1) & (data6_1["reg_raw"]<=1)]))
print(len(mollib_1[(mollib_1["clf_ex1_bin"]==1) & (mollib_1["reg_raw"]<=1)]))

1118
202


In [35]:
#delete molecules that are predicted inactive with cut off 2.5 but active with cut off 1
data6_25_actives = data6_25[(data6_25["clf_ex1_bin"]==1) & (data6_25["reg_raw"]<=2.5)]
data6_1_actives = data6_1[(data6_1["clf_ex1_bin"]==1) & (data6_1["reg_raw"]<=1)]
data6_1_actives.reset_index(inplace=True)
mollib_25_actives = mollib_25[(mollib_25["clf_ex1_bin"]==1) & (mollib_25["reg_raw"]<=2.5)]
mollib_1_actives = mollib_1[(mollib_1["clf_ex1_bin"]==1) & (mollib_1["reg_raw"]<=1)]
mollib_1_actives.reset_index(inplace=True)


In [36]:
data6_25_smi = data6_25_actives["smiles"].tolist()
data6_1_smi = data6_1_actives["smiles"].tolist()

idx = []
for i,smi in enumerate(data6_1_smi):
    if smi not in data6_25_smi:
        idx += [i]

print(len(data6_1_actives))       
data6_1_actives.drop(index=idx, axis = 0, inplace=True)
print(len(data6_1_actives))

1118
1094


C:\Users\Gemma\Anaconda3\envs\malariaOSM\lib\site-packages\pandas\core\frame.py:4315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [37]:
mollib_25_smi = mollib_25_actives["smiles"].tolist()
mollib_1_smi = mollib_1_actives["smiles"].tolist()

idx = []
for i,smi in enumerate(mollib_1_smi):
    if smi not in mollib_25_smi:
        idx += [i]

print(len(mollib_1_actives))       
mollib_1_actives.drop(index=idx, axis = 0, inplace=True)
print(len(mollib_1_actives))

202
0
